In [45]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        # Filter out directories and files starting with '.'
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        files = [f for f in files if not f.startswith('.')]
        
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

# Set the startpath to the current directory (repo root)
startpath = '.'

# Run the function
list_files(startpath)


./
    go.mod
    go.sum
    README.md
    package-lock.json
    package.json
    AIDevops.ipynb
    dist/
        schema.json
    samples/
        step-template.yaml
        step.yaml
        stage-group.yaml
        step-approval.yaml
        stage-runtime.yaml
        failure-strategy-retry.yaml
        step-background.yaml
        pipeline-clone.yaml
        step-run.yaml
        pipeline-inputs.yaml
        step-parallel.yaml
        failure-strategy-manual.yaml
        output.yaml
        stage-template.yaml
        step-run-test.yaml
        matrix.yaml
        step-queue.yaml
        step-action.yaml
        step-barrier.yaml
        stage-volumes.yaml
        pipeline-on.yaml
        failure-strategy.yaml
        pipeline-service.yaml
        stage-caching.yaml
        pipeline.yaml
        stage-approval.yaml
        environment.yaml
        stage.yaml
        step-group.yaml
        stage-parallel.yaml
        complex/
            dynamic-provisioned-infra.yaml
            k

In [ ]:
!pip install openai

In [66]:
import os
import openai
from openai import OpenAI

# Load the OpenAI API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

def gather_repo_context(folders):
    context = ""
    file_list = []
    for folder in folders:
        for root, dirs, files in os.walk(folder):
            dirs[:] = [d for d in dirs if not d.startswith('.')]  # Exclude hidden directories
            files = [f for f in files if not f.startswith('.')]  # Exclude hidden files
            
            for file in files:
                if file.endswith(('.yaml', '.yml', '.ts', '.js', '.json', '.md', '.go')):
                    file_path = os.path.join(root, file)
                    file_list.append(file_path)
                    with open(file_path, 'r') as f:
                        context += f"\n\nFile: {file_path}\n" + f.read()
    return context, file_list

def create_messages(task, prompt, current_pipeline=None, logs=None, error_message=None, repo_context="", secrets=None, user_info=None):
    system_prompt = (
        "You are an AI programming assistant named 'AI Devops Engineer'. "
        "You are part of a chat interface for an open source CI/CD platform called 'Gitness'. "
        "Follow the user's requirements carefully & to the letter. "
        "Your expertise is strictly limited to CI/CD and DevOps topics. "
        "Follow Harness content policies. "
        "Avoid content that violates copyrights. "
        "For questions not related to CI/CD and DevOps, simply give a reminder that you are an AI Devops Engineer. "
        "Keep your answers short and impersonal. "
        "You can answer general DevOps questions and perform the following tasks through tool calls: "
        "* Generate a CI/CD pipeline configuration "
        "* Update a CI/CD pipeline configuration "
        "* Analyze failed pipeline logs and suggest a code fix "
        "First think step-by-step - describe your plan for what to build and then do it. "
        "Minimize any other prose. "
        "Avoid wrapping the whole response in triple backticks. "
        "You can understand and use context from Gitness examples, JSON schema, and YAML files. "
        "Each time you respond ensure the USER QUERY is satisfied to the best of your ability. "
        "Make no assumptions about the functions and usage of YAML files that do not exist in the context, if there is missing information, ask for that information. "
        "Do not make up an answer to questions that are not related to the query. "
        "Prioritize using tool calls to perform the tasks when possible. "
        "Do not explicitly mention the tool in the response. "
        "There is no need to provide the full code or yaml in the message response unless specifically asked for. "
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "system", "content": f"Context from the repo: {repo_context}"}
    ]
    
    if secrets:
        messages.append({"role": "system", "content": f"List of secrets: {secrets}"})
    
    if user_info:
        messages.append({"role": "system", "content": f"User information: {user_info}"})
    
    if task == "generate_pipeline":
        messages.append({"role": "user", "content": prompt})
    elif task == "update_pipeline":
        if current_pipeline is None:
            raise ValueError("current_pipeline must be provided for updating a pipeline.")
        messages.append({"role": "user", "content": f"The current pipeline configuration is:\n{current_pipeline}\n{prompt}"})
    elif task == "fix_failed_pipeline":
        if logs is None or error_message is None:
            raise ValueError("logs and error_message must be provided for analyzing a failed pipeline.")
        messages.append({"role": "user", "content": f"Analyze the following pipeline logs and error message to suggest a code fix:\nLogs:\n{logs}\nError Message:\n{error_message}"})
    else:
        raise ValueError("Unknown task. Please provide a valid task: generate_pipeline, update_pipeline, or fix_failed_pipeline.")
    
    return messages

def create_tools(task):
    if task == "generate_pipeline":
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "generate_pipeline",
                    "description": "Generates a CI/CD pipeline.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "pipeline_config": {
                                "type": "string",
                                "description": "The YAML configuration of the generated pipeline."
                            }
                        },
                        "required": ["pipeline_config"]
                    }
                }
            }
        ]
    elif task == "update_pipeline":
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "update_pipeline",
                    "description": "Updates a CI/CD pipeline",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "updated_pipeline_config": {
                                "type": "string",
                                "description": "The updated YAML configuration of the pipeline."
                            }
                        },
                        "required": ["updated_pipeline_config"]
                    }
                }
            }
        ]
    elif task == "fix_failed_pipeline":
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "fix_failed_pipeline",
                    "description": "Fix a failed pipeline.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "suggested_fix": {
                                "type": "string",
                                "description": "The suggested code fix based on the logs and error message."
                            }
                        },
                        "required": ["suggested_fix"]
                    }
                }
            }
        ]
    else:
        raise ValueError("Unknown task. Please provide a valid task: generate_pipeline, update_pipeline, or fix_failed_pipeline.")
    
    return tools

def call_openai_function(task, prompt, current_pipeline=None, logs=None, error_message=None, temperature=0.3, seed=42):
    folders = ['./samples', './dist']  # List of folders to gather context from
    repo_context, file_list = gather_repo_context(folders)

    secrets = {
        "AWS_ACCESS_KEY_ID": "AKIA...",
        "AWS_SECRET_ACCESS_KEY": "abc123...",
        "DOCKER_PASSWORD": "s3cr3t..."
    }
    
    user_info = {
        "username": "johndoe",
        "email": "johndoe@example.com",
        "role": "developer"
    }
    
    messages = create_messages(task, prompt, current_pipeline, logs, error_message, repo_context, secrets, user_info)
    tools = create_tools(task)
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        temperature=temperature,
        user="userId",  # https://platform.openai.com/docs/guides/safety-best-practices/end-user-ids - not required, but recommended
        seed=seed,
        tool_choice="required"

    )

    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            print("Made tool call:")
            print(f"Tool Name: {tool_call.function.name}")
            print(f"Arguments: {tool_call.function.arguments}")
    else:
        print("No tool call made.")

    return response.choices[0].message.content


In [70]:
# Example 1: Generate a CI/CD Pipeline Configuration
task = "generate_pipeline"
prompt = "Build python build pipeline"
response = call_openai_function(task, prompt)
print(response)


Made tool call:
Tool Name: generate_pipeline
Arguments: {"pipeline_config":"pipeline:\n  stages:\n    - steps:\n        - run:\n            script: |\n              python setup.py install\n              python -m unittest discover"}
None


In [69]:

# Example 2: Update a CI/CD Pipeline Configuration
task = "update_pipeline"
current_pipeline = """
pipeline:
  stages:
    - name: Lint
      steps:
        - name: Run Flake8
          image: python:3.8
          commands:
            - pip install flake8
            - flake8 .
    - name: Test
      steps:
        - name: Run Tests
          image: python:3.8
          commands:
            - pip install -r requirements.txt
            - pytest
"""
prompt = "Add a stage for deploying to Google Cloud Platform (GCP) using the user's credentials. Update it for me."
response = call_openai_function(task, prompt, current_pipeline=current_pipeline)
print(response)

Made tool call:
Tool Name: update_pipeline
Arguments: {"updated_pipeline_config":"pipeline:\n  stages:\n    - name: Lint\n      steps:\n        - name: Run Flake8\n          image: python:3.8\n          commands:\n            - pip install flake8\n            - flake8 .\n    - name: Test\n      steps:\n        - name: Run Tests\n          image: python:3.8\n          commands:\n            - pip install -r requirements.txt\n            - pytest\n    - name: Deploy\n      steps:\n        - name: Authenticate GCP\n          image: google/cloud-sdk:latest\n          commands:\n            - echo $GOOGLE_APPLICATION_CREDENTIALS > /tmp/account.json\n            - gcloud auth activate-service-account --key-file=/tmp/account.json\n        - name: Deploy to GCP\n          image: google/cloud-sdk:latest\n          commands:\n            - gcloud app deploy"}
None


In [67]:

# Example 3: Analyze Failed Pipeline Logs and Suggest a Fix
task = "fix_failed_pipeline"
logs = """
Pipeline execution failed at stage 'Build'. Error parsing YAML configuration:
- name: Build
  steps:
    - name: Build Image
      image: docker:latest
      commands:
        - docker build -t my-app .
        - docker push my-app:latest
      environment:
      - DOCKER_PASSWORD=${{ secrets.DOCKER_PASSWORD }}
      - DOCKER_USERNAME=${{ secrets.DOCKER_USERNAME }}
[ERROR] Invalid YAML format at line 6, column 7: 'environment' should be a map, but found a sequence instead.
"""
error_message = "YAML configuration error: 'environment' should be a map, but found a sequence instead."

response = call_openai_function(task, None, logs=logs, error_message=error_message)
print(response)

Made tool call:
Tool Name: fix_failed_pipeline
Arguments: {"suggested_fix":"Modify the 'environment' section to be a map instead of a sequence. Change:\n  environment:\n  - DOCKER_PASSWORD=${{ secrets.DOCKER_PASSWORD }}\n  - DOCKER_USERNAME=${{ secrets.DOCKER_USERNAME }}\nTo:\n  environment:\n    DOCKER_PASSWORD: ${{ secrets.DOCKER_PASSWORD }}\n    DOCKER_USERNAME: ${{ secrets.DOCKER_USERNAME }}"}
None


# Start testing against HTTP endpoint

In [78]:
import requests
import json

# Base URL for the FastAPI app
base_url = "http://localhost:8001"

# Example 1: Generate a CI/CD Pipeline Configuration
def example_generate_pipeline():
    url = f"{base_url}/chat"
    payload = {
        "prompt": "Build python build pipeline"
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 1 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Example 2: Update a CI/CD Pipeline Configuration
def example_update_pipeline():
    url = f"{base_url}/chat"
    current_pipeline = """
    pipeline:
      stages:
        - name: Lint
          steps:
            - name: Run Flake8
              image: python:3.8
              commands:
                - pip install flake8
                - flake8 .
        - name: Test
          steps:
            - name: Run Tests
              image: python:3.8
              commands:
                - pip install -r requirements.txt
                - pytest
    """
    prompt = "Add a stage for deploying to Google Cloud Platform (GCP) using the user's credentials. Update it for me."
    payload = {
        "prompt": prompt,
        "extra_context": [
            {
                "description": "Current Pipeline",
                "content": current_pipeline
            }
        ]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 2 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Example 3: Analyze Failed Pipeline Logs and Suggest a Fix
def example_fix_failed_pipeline():
    url = f"{base_url}/chat"
    logs = """
    Pipeline execution failed at stage 'Build'. Error parsing YAML configuration:
    - name: Build
      steps:
        - name: Build Image
          image: docker:latest
          commands:
            - docker build -t my-app .
            - docker push my-app:latest
          environment:
          - DOCKER_PASSWORD=${{ secrets.DOCKER_PASSWORD }}
          - DOCKER_USERNAME=${{ secrets.DOCKER_USERNAME }}
    [ERROR] Invalid YAML format at line 6, column 7: 'environment' should be a map, but found a sequence instead.
    """
    prompt = "YAML configuration error: 'environment' should be a map, but found a sequence instead."
    payload = {
        "prompt": prompt,
        "extra_context": [
            {
                "description": "Pipeline Logs",
                "content": logs
            }
        ]
    }
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Example 3 Response:")
        print(response.json())
    else:
        print("Error:", response.status_code, response.text)

# Run examples
if __name__ == "__main__":
    example_generate_pipeline()
    example_update_pipeline()
    example_fix_failed_pipeline()


Example 1 Response:
{'response': {'content': "I will create a CI/CD pipeline to build a Python project. The pipeline will include the necessary steps to install dependencies, run tests, and build the application. Here is the step-by-step plan:\n\n1. **Install dependencies**:\n   - Typically, dependencies are listed in a `requirements.txt` or `Pipfile`.\n2. **Run tests**:\n   - Use a test framework like `unittest`, `pytest`, etc.\n3. **Build the application**:\n   - For Python, this may mean packaging the application for distribution.\n\nLet's proceed with creating the YAML configuration for this pipeline.\n\n```yaml\npipeline:\n  stages:\n    - name: Install Dependencies\n      steps:\n        - run:\n            script: |\n              pip install -r requirements.txt\n\n    - name: Run Tests\n      steps:\n        - run:\n            script: |\n              pytest\n\n    - name: Build Package\n      steps:\n        - run:\n            script: |\n              python setup.py sdist\n